# Actividad ETL con Python 🐍 y Postgres 🐘

- Realizar la unión de contenido de 2 archivos, con la informacion de la contaminacion del aire de 8 ciduad entre 2017 y 2020.

## Importacion de librerias

In [ ]:
# Importacion de librerias
import pandas as pd
import datetime
import numpy
from sqlalchemy import create_engine
import psycopg2

import sys

In [ ]:
# Guardamos en variables la ubicacion de los archivos
path_pm = 'NivelesContaminacion_pm25.csv'
path_o = 'NivelesContaminacion_o3.csv'

# Leemos el contenido de los archivos csv
data_pm = pd.read_csv(path_pm)
data_o = pd.read_csv(path_o)

In [ ]:
# Se lee el contenido
data_pm.head()

In [ ]:
# Se lee el contenido
data_o.head()

## Eliminacion de columnas para ```data_o```

In [ ]:
# Se eliminan las columnas 5, 6 y 7
# axis = 1 => especifica que es la columna
data_o.drop(data_o.columns[[5, 6, 7]], axis=1, inplace=True)

In [ ]:
# Se verifica el resultado
data_o

## Renombrar columnas para ```data_o```

In [ ]:
# Se renombrará la columnas <count> por <count_o3>
data_o.columns = ["Date", "Country", "City", "Specie", "Count_o3"]

In [ ]:
# Se verifica el resultado
data_o.columns

## Más eliminaciones de columna para ```data_o```

In [ ]:
# Eliminacion de la columna 3 "Specie"
data_o.drop(data_o.columns[[3]], axis=1, inplace=True)

In [ ]:
# Se verifica el resultado
data_o.columns

## Eliminación de columnas para ```data_pm```

In [ ]:
# Se eliminan las columnas 5, 6 y 7
# axis = 1 => especifica que es la columna
data_pm.drop(data_pm.columns[[5, 6, 7]], axis=1, inplace=True)

In [ ]:
# Se verifica el resultado
data_pm

## Renombrar columnas para ```data_pm```

In [ ]:
# Se renombrará la columna <count> por <count_pm>
data_pm = data_pm.rename({'count': 'Count_pm25'}, axis=1)

# Se elimina la columna 3 "Specie"
data_pm.drop(data_pm.columns[[3]], axis=1, inplace=True)

In [ ]:
# Se verifica el resultado
data_pm.columns

## Combinar los datas

In [ ]:
data_merged_pm_o = pd.merge(data_o,
                            data_pm,
                            how='left',
                            left_on=["Date", "Country", "City"],
                            right_on=["Date", "Country", "City"]
                            )

In [ ]:
# Se verifica el resultado
data_merged_pm_o

## Carga de datos

- BD : postgre
- Database : etl_with_python

In [ ]:
# Se comprueban los campos de la nueva coleccion
data_merged_pm_o.columns

### Query de creacion de tabla

```sql
CREATE TABLE count_o3_pm25 (
  "date" DATE,
  "country" VARCHAR,
  "city" VARCHAR,
  "count_o3" INT,
  "count_pm25" INT
)
```

## Conexión a postgres

In [ ]:
# Declaramos un diccionario con datos para la conexion a la bd
postgre_dictionary_connection = {
  "host": "localhost",
  "user": "postgres",
  "password": "pato",
  "database": "etl_with_python"
}

In [ ]:
# Declaramos la funcion para conectar a postgre
def connect_to_postgre(dictionary_connection):
    connection = None

    try:
        print('Connecting to PostgreSQL server')
        connection = psycopg2.connect(**dictionary_connection)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)

    print("Connection successful")
    return connection

In [ ]:
# Guardamos en <connection> la conexion a postgree
connection = connect_to_postgre(postgre_dictionary_connection)

In [ ]:
# Definicion una funcion para insertar datos en postgree
def execute_many(connection, data, table):
    tuples = [tuple(x) for x in data.to_numpy()]

    cols = ','.join(list(data.columns))

    query = "INSERT INTO %s(%s) VALUES (%%s, %%s, %%s, %%s, %%s)" % (
        table, cols)

    cursor = connection.cursor()

    try:
        cursor.executemany(query, tuples)
        connection.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        connection.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

In [ ]:
# Ejecutamos la funcion
execute_many(connection, data_merged_pm_o, 'count_o3_pm25')